<a href="https://colab.research.google.com/github/apsadewa/Steganography/blob/main/Copy_of_RDH_4R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import

In [39]:
!pip install img2pdf
!pip install pdf2img
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [40]:
!sudo apt install tesseract-ocr libtesseract-dev libleptonica-dev pkg-config
!pip install tesserocr
!pip install db-sqlite3
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pkg-config is already the newest version (0.29.2-1ubuntu3).
libleptonica-dev is already the newest version (1.82.0-3build1).
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [41]:
# from image_slicer import slice
import cv2
from google.colab.patches import cv2_imshow
import pandas as pd
import numpy as np
from math import log10, sqrt, floor
from PIL import Image
import matplotlib.image as mpimg
import hashlib
import uuid
from numba import njit
from numba.typed import List
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import os
import time
import sqlite3
import pandas as pd
from google.colab import drive

path = "/content/drive/MyDrive/TA/Code/Dataset/Pengujian/"
realpath = "/content/drive/MyDrive/TA/Code/Dataset/Pengujian/Refrensi/"
data_url = "https://docs.google.com/spreadsheets/d/1dWyK1rubUljef-sdnRs2xb5I6s9GCZFKSkWHiBpsC5Q/edit?usp=sharing"


# Embed Function

In [47]:
import sqlite3

def createDB(dbname):
    # Connect to the database
    db = sqlite3.connect(dbname + ".db")

    # Drop the table if it exists
    db.execute("DROP TABLE IF EXISTS validation")

    # Create the table
    try:
        db.execute("CREATE TABLE validation(name TEXT, seed TEXT)")
        print(f"Table 'validation' created successfully in {dbname}.db")
    except Exception as e:
        print(f"Error creating table: {e}")

    # Commit the changes and close the connection
    db.commit()
    db.close()

def insertData(name, seed):
    # Connect to the database
    db = sqlite3.connect("val.db")

    # Check if the name already exists
    cek = "SELECT name FROM validation WHERE name=?"
    rs = db.execute(cek, (name,)).fetchall()

    # If the name does not exist, insert a new record
    if len(rs) == 0:
        cmd = "INSERT INTO validation(name, seed) VALUES(?, ?)"
        db.execute(cmd, (name, seed))
    # If the name exists, update the existing record
    else:
        cmd = "UPDATE validation SET seed = ? WHERE name = ?"
        db.execute(cmd, (seed, name))

    # Commit the changes and close the connection
    db.commit()
    db.close()

# Create the database and table
createDB("val")

Table 'validation' created successfully in val.db


In [48]:
@njit
def toModthree(value):
  if (value%3 == 1):
    value-=1
  elif(value%3 == 2):
    value+=1
  return value

In [49]:
def SubBlockImage(img, height, width):
  block = []
  for i in range(height):
    k=0
    for j in range(3,width+3, 3): # Membagi setiap baris menjadi blok-blok dengan lebar 3
      block.append(img[i,k:j])
      k+=3
  block = np.asarray(block)
  return block

In [50]:
@njit(fastmath=True)
def binarySearch (arr, l, r, x):
    if r >= l:
        mid = l + (r - l) // 2
        if arr[mid] == x:
            return True
        elif arr[mid] > x:
            return binarySearch(arr, l, mid-1, x)
        else:
            return binarySearch(arr, mid + 1, r, x)
    else:
        return False

@njit(fastmath=True)
def calculateBlock(pixels):
  s=0
  for i in range(len(pixels)):
    s+=pixels[i]
  return int(s*100)

@njit(fastmath=True)
def OverUnder_Flow_Handle(block, idx):
  L=[]
  S=[]
  for j in range(len(block)):
    k = 0
    p=0
    flag = binarySearch(idx, 0, len(idx)-1, j)
    for i in range(9):

      if(i%3==0 and i!=0):
        k+=1
      if flag:
        p+=block[j,k,i%3]/255

      if(block[j,k,i%3] == 0):
        block[j,k,i%3] = 1
        L.append(1)

      elif(block[j,k,i%3] == 255):
        block[j,k,i%3] = 254
        L.append(1)

      elif(block[j,k,i%3] == 1 or block[j,k,i%3] == 254):
        L.append(0)

    if flag:
      S.append(int(p*100))
  return L, S, block

In [51]:
def generateRandInd(lenSubBlock, seed=None):
  idx = []
  if not seed:
    seed = int(np.random.uniform(0,9999))
  rng = np.random.default_rng(seed)
  for i in range(lenSubBlock):
    if(rng.random() > 0.99):
      idx.append(i)
  return List(idx), seed

def generateRandEmb(ids, lenSubBlock, seed):
  print("MASUK KE RANDEMBD")
  print(ids, lenSubBlock, seed)
  rng = np.random.default_rng(seed)
  a = [i for i in range(ids+1,lenSubBlock)]
  print("ids {}, len {}, seed {}".format(ids, lenSubBlock, seed))
  idx = rng.choice(a,len(a), replace=False)
  print(idx)
  return List(idx)

  #ada index buat di sum
  #ad index buat embed

In [52]:
@njit
def differencePixel(C1, subBlocks):
  C1 = C1.astype(np.int16)
  for i in range(len(C1)):
    k = 0
    x = subBlocks[i,k+1,1]
    for j in range(9):
      # print(i, k , j)
      if(j%3==0 and j!=0):
        k+=1
      if(j%3==1 and k==1):
        C1[i,k,j%3] = x
      else:
        C1[i,k,j%3] -= x
  return C1

In [53]:
def InData(name, S, L):
  strS = ''.join([str(bin(x))[2:].zfill(10) for x in S])
  strL = ''.join(map(str,L))
  sep = ''.join(format(i, '08b') for i in bytearray("|", encoding ='utf-8')).zfill(10) + ''.join(format(i, '08b') for i in bytearray(">", encoding ='utf-8')).zfill(10)
  print("sep:", sep)
  data =  name +"|"

  bindata = ''.join(format(i, '08b') for i in bytearray(data, encoding ='utf-8'))

  mergedata = bindata + strS + sep + strL
  mergedataInt = List(list(map(int, [bit for bit in mergedata])))
  return mergedataInt

def stringTobit(string):
  bindata = ''.join(format(i, '08b') for i in bytearray(string, encoding ='utf-8'))
  return list(map(int, [bit for bit in bindata]))


In [ ]:
@njit
def embeddingImg(C1, D, data, pr, pl):
    counterData = 0

    for idx in range(len(subBlock)):
        i = idx // C1.shape[1]  # Menghitung indeks i
        k = idx % C1.shape[1]  # Menghitung indeks k

        for j in range(9):  # Iterasi melalui 9 elemen yang mungkin dalam dimensi ketiga
            if j % 3 == 0 and j != 0:
                k += 1

            if j % 3 == 1 and k == 1:
                C1[i, k, j % 3] += 0
            elif counterData < len(data):
                if D[i, k, j % 3] < pl:
                    C1[i, k, j % 3] -= 1
                elif D[i, k, j % 3] == pl:
                    C1[i, k, j % 3] -= data[counterData]
                    counterData += 1
                elif D[i, k, j % 3] == pr:
                    C1[i, k, j % 3] += data[counterData]
                    counterData += 1
                elif D[i, k, j % 3] > pr:
                    C1[i, k, j % 3] += 1
            elif counterData >= len(data):
                if D[i, k, j % 3] <= pl:
                    C1[i, k, j % 3] -= 1
                elif D[i, k, j % 3] >= pr:
                    C1[i, k, j % 3] += 1
                counterData += 1

    return C1


In [35]:
def stackImage(C2_Block, height, width):
  index1 = 0
  index2 = width//3
  z = list()
  for i in range(height):
    z.append(np.reshape(C2_Block[index1:index2],(width,3)))
    index1 = index2
    index2 += width//3
  return np.array(z)

In [36]:
@njit
def PSNR(original, embedded):
    mse = np.mean((original - embedded) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [37]:
def Embedding(img, name):
    height, width = img.shape[0], img.shape[1]
    height, width = toModthree(height), toModthree(width)
    img = cv2.resize(img, (width, height))
    print("Shape", img.shape)

    subBlock = SubBlockImage(img, height, width)
    Lidx, seed = generateRandInd(len(subBlock), seed=1234)
    print("restidx: ", len(Lidx), Lidx)
    print("seed awal : ", seed)
    mapL, mapS, C = OverUnder_Flow_Handle(subBlock, Lidx)
    data = InData(name, mapS, mapL)
    print("Len Block : ",len(subBlock))
    print("Data :", len(data), data)
    print("seed:", seed)
    print("len sub: ", len(subBlock))
    print("mapS :", len(mapS), mapS)
    print("mapL :", len(mapL), mapL)


    subCopy = C.copy()
    D = differencePixel(subCopy, C)
    plt.hist(D.ravel(), 256, [0, 256])
    plt.show()
    pr, pl = getpeakvalue(D, data)
    print(pr, pl)
    pp = ""

    if pr < 0:
        pr_fix = abs(pr)
        pl_fix = abs(pl)
        pp += "11"
    elif pl < 0:
        pl_fix = abs(pl)
        pr_fix = pr
        pp += "01"
    else:
        pr_fix = pr
        pl_fix = pl
        pp += "00"

    pp += format(pr_fix, "b").zfill(8)
    pp += format(pl_fix, "b").zfill(8)

    pp = list(map(int, pp))
    data = list(data)
    print("List Data :", data)
    embedding_data = List(pp + data)
    print("pp : ", pp)

    # Panggil fungsi embedding hanya dengan lima argumen
    C2 = embeddingImg(C.copy(), D, embedding_data, pr, pl)
    embed_image = stackImage(C2, height, width)

    return cv2.imwrite("Embedded.png", embed_image)



def getpeakvalue(difblock, data):
  unique, counts = np.unique(difblock, return_counts=True)
  dicts = dict(zip(unique, counts))
  #print("dblock: ",difblock)
  print("dicts:", dicts)
  getTwopeak = sorted(((v,k) for k,v in dicts.items()))[-2:] #membuat generator yang menghasilkan pasangan (frekuensi, nilai) dan mengurutkan pasangan berdasarkan frekuensi dalam urutan menaik dan mengambil dua elemen terakhir (dua nilai dengan frekuensi tertinggi)
  if(getTwopeak[0][1] > getTwopeak[1][1]): #menetapkan pr (nilai puncak tertinggi) dan pl (nilai puncak terendah)
    pr = getTwopeak[0][1]
    pl = getTwopeak[1][1]
  else:
    pr = getTwopeak[1][1]
    pl = getTwopeak[0][1]
  print(getTwopeak[0][0],getTwopeak[0][1], getTwopeak[1][0], getTwopeak[1][1],"pr: ", pr, "pl :",pl)
  return pr, pl


# Run Embed

In [38]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive',force_remount=True)

# Tetapkan path
realpath = "/content/drive/MyDrive/TA/Code/Dataset/Pengujian/Refrensi"
subdir = realpath + "/"

# Periksa apakah path ada
if os.path.exists(subdir):
    try:
        # Ambil dan sortir daftar file
        imgList = sorted(os.listdir(subdir), key=lambda x: int(x.split('.')[0]))
        print("List of images:", imgList)
    except Exception as e:
        print(f"Error accessing directory: {e}")
else:
    print(f"Subdirectory does not exist: {subdir}")

    # Daftar isi direktori utama untuk debugging
    if os.path.exists(realpath):
        print("Contents of the main directory:")
        print(os.listdir(realpath))
    else:
        print(f"Main directory does not exist: {realpath}")


KeyboardInterrupt: 

In [ ]:
img = cv2.imread("/content/drive/MyDrive/TA/Code/Dataset/Pengujian/Refrensi/1.jpg")

width, height = 1280, 720
img = cv2.resize(img, (width,height))

In [ ]:
cv2_imshow(img)

In [ ]:
name = "Xebelion"

%time Embedding(img, name)

In [ ]:
cv2_imshow(cv2.imread("/content/Embedded.png"))

In [ ]:
!ls -s

# Extraction


